<a href="https://colab.research.google.com/github/tam1444AH/COSC4397Project/blob/main/notebooks/supervised-data-preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install nbstripout
%pip install -U "huggingface-hub>=0.34.0,<1.0"
%pip check
%pip install hf_transfer
%pip install -U bitsandbytes --upgrade
%pip install transformers datasets
%pip install transformers datasets peft flash-attn
!export HF_HUB_ENABLE_HF_TRANSFER=1

from google.colab import auth
auth.authenticate_user()
import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
import os
import wandb
import shutil
from datetime import datetime
from google.colab import userdata
from huggingface_hub import login, whoami
os.environ["WANDB_DISABLED"] = "false"  # or "true" to mute
os.environ["WANDB_PROJECT"]   = "qwen3coder-finetune-fp16"

os.environ["WANDB_LOG_MODEL"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
REPO_URL="https://github.com/UH-Insure/Finetuning-Qwen3.git"
REPO="Finetuning-Qwen3"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    %pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    %pip install -e .

In [ ]:
!pip -q install -U "transformers>=4.43" "datasets>=2.20" peft bitsandbytes trl wandb

import os, math, json, random, torch
from time import time
from datetime import datetime
import wandb

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
)
from trl import SFTTrainer
from peft import PeftModel